In [ ]:
import nltk
import os
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import tkinter as tk
from tkinter import scrolledtext

inverted_index = {}
global entry_query
global result_box_inverted
global result_box_positional# Define result_box globally

stop_words = {
    'a', 'is', 'the', 'of', 'all', 'and', 'to', 'can', 'be', 'as',
    'once', 'for', 'at', 'am', 'are', 'has', 'have', 'had', 'up', 'his',
    'her', 'in', 'on', 'no', 'we', 'do'
}

single_aplha = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
         'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'}

target_chars = ['%','$','*',"'",'’','¨','=','+','`','/' , '.' ,'·', ',', '0','1' , '2', '3' , '4' , '5', '6' , '7' , '8', '9' ,'|' ,':' ,'(' ,')','>',';','&', '“','”','[',']','@','?','}','{']  # Character you want to replace

operators = {'and' , 'or' , 'not'}

def Pre_Processing(input_file, output_file):
    
        with open(input_file, 'r',encoding='utf-8', errors='ignore') as file:
            content = file.read()
            updated_content = ''
          
            for char in content:
                if char in target_chars:
                    updated_content += ' '  #Replace the character with a space
                elif char == '-' and '_' :
                    updated_content += ''
                elif char == '\n':
                    updated_content += ' '  
                else:
                    updated_content += char  #Keep the character unchanged
                       
            
        with open(output_file, 'w') as file:
            file.write(updated_content)
     
        print(f"File {input_file} Pre-Processed successfully!")


def tokenize_and_stemm_doc_and_inserting_terms_in_Inverted_index(filename, stem_store, doc_id):

    with open(filename, 'r') as file:
        content = file.read()

    # Tokenizing
    tokens = word_tokenize(content)

    # Porter Stemmer
    porter_stemmer = PorterStemmer()

    stemmed_tokens = []

    for token in tokens:
        if token.lower() not in stop_words and token not in single_aplha:
            stemmed_word = porter_stemmer.stem(token)
            stemmed_tokens.append(stemmed_word)


    with open(stem_store, 'w') as file:
        for token in stemmed_tokens:
            file.write(token + '\n')
    print(f"File {filename} tokenized, stemmed successfully!")

    # Insert each stemmed-token(sorted) in inverted index
    for token in sorted(stemmed_tokens):
        if token not in inverted_index:  # If token not in inverted index
            inverted_index[token] = [doc_id]  # Add it
        else:
            if doc_id not in inverted_index[token]:  # If token is already inserted, check if its posting list has the current document-id
                # Find the correct position to insert the document-id to maintain lexicographic order
                index = 0
                while index < len(inverted_index[token]) and inverted_index[token][index] < doc_id:
                    index += 1
                inverted_index[token].insert(index, doc_id)

    print(f"File {stem_store} inserted in inverted index successfully!")

    
def tokenize_and_stemm_doc_and_inserting_terms_in_Positional_Inverted_index(filename, stem_store, doc_id):
    
    with open(filename, 'r') as file:
        content = file.read()
    
    porter_stemmer = PorterStemmer()
    
    tokens = word_tokenize(content)
    
    current = 0
    stemmed_tokens_with_position = []
    for token in tokens:
        if token.lower() not in stop_words and token not in single_aplha:
            stemmed_token = porter_stemmer.stem(token)
            stemmed_tokens_with_position.append((stemmed_token , current))
            current += len(token) + 1
        else:
            current += len(token) + 1
       
    with open(stem_store, 'w') as file:
        for token in stemmed_tokens_with_position:
            file.write(str(token) + '\n')
    print(f"File {filename} tokenized, stemmed successfully!")
    
    
    
    for token, position in sorted(stemmed_tokens_with_position):
        if token not in inverted_index:
            to_insert = {doc_id : [position]}
            inverted_index[token] = [to_insert]
        else:
            exist = False
            for item in inverted_index[token]:
                if doc_id in item:
                    exist = True
                    item[doc_id].append(position)
                
                    
            if not exist:
                to_insert = {doc_id : [position]}
                inverted_index[token].append(to_insert)
            
           
    print(f"File {stem_store} inserted in inverted index successfully!")   

    
    
def Pre_Processing_query(query):
    
    updated_query = ''
    for char in query:
        if char in target_chars:
            updated_query += ' '  #Replace the character with a space
        elif char == '-' or char == '_':
            updated_query += ''  
        else:
            updated_query += char  #Keep the character unchanged
    return updated_query
   
def intersect(p1, p2):
    answer = []
    i, j = 0, 0
    
    while i < len(p1) and j < len(p2):
        if p1[i] == p2[j]:
            answer.append(p1[i])
            i += 1
            j += 1
        elif p1[i] < p2[j]:
            i += 1
        else:
            j += 1
    
    return answer



def union(p1 , p2):
    return list(set(p1).union(set(p2)))

def not_operation(p1):
    list1 = [1,2,3,7,8,9,11,12,13,14,15,16,17,18,21,22,23,24,25,26]
    return list(set(list1).difference(set(p1)))

def query_processing(entry_query):  # Accept entry_query as argument
    global result_box_inverted  # Access result_box globally
    query = entry_query.get()  # Access entry_query using the argument
    pp_query = Pre_Processing_query(query)
    print(pp_query)
    
    porter_stemmer = PorterStemmer()
    
    words = pp_query.split()
    stemmed_query = []
    
    for word in words:
        stemmed_query.append(porter_stemmer.stem(word))

    print(stemmed_query)
   
 
    stack = []
    i=0
    while i < len(stemmed_query):
        if stemmed_query[i] not in operators:
            if(stemmed_query[i] in inverted_index):
                stack.append(inverted_index[stemmed_query[i]])
                i +=1
            else:
                print('term not in inverted index')
                return
        elif stemmed_query[i] == 'not' and i+1 < len(stemmed_query):
            if(stemmed_query[i+1] in inverted_index):
                stack.append(not_operation(inverted_index[stemmed_query[i+1]]))
                i += 2
            else:
                print('term not in inverted index')
                return    
        elif stemmed_query[i] == 'and'and i+1 < len(stemmed_query):
            op1 = stack.pop()
            if stemmed_query[i+1] != 'not':
                if(stemmed_query[i+1] in inverted_index):
                    result = intersect(op1 , inverted_index[stemmed_query[i+1]] )
                    stack.append(result)
                    i += 2
                else:
                    print('term not in inverted index')
                    return    
            elif stemmed_query[i+1] == 'not'and i+2 < len(stemmed_query):
                if(stemmed_query[i+2] in inverted_index):
                    result = intersect(op1 ,not_operation(inverted_index[stemmed_query[i+2]]))
                    stack.append(result)
                    i += 3
                else:
                    print('term not in inverted index')
                    return     
            else:
                print('invalid query')
                return    
        elif stemmed_query[i] == 'or'and i+1 < len(stemmed_query):
            op1 = stack.pop()
            if stemmed_query[i+1] != 'not':
                if(stemmed_query[i+1] in inverted_index):
                    result = intersect(op1 , inverted_index[stemmed_query[i+1]] )
                    stack.append(result)
                    i += 2
                else:
                    print('term not in inverted index')
                    return    
            elif stemmed_query[i+1] == 'not'and i+2 < len(stemmed_query):
                if(stemmed_query[i+2] in inverted_index):
                    result = intersect(op1 ,not_operation(inverted_index[stemmed_query[i+2]]))
                    stack.append(result)
                    i += 3 
                else:
                    print('term not in inverted index')
                    return     
            else:
                print('invalid query')
                return
        else:
                print('invalid query')
                return    
            
    if(len(stack) == 1):
        result_box_inverted.delete(1.0, tk.END)  # Clear previous result
        result_box_inverted.insert(tk.END,str(stack.pop()) )
    else:
        print(stack)

def proximity_query(entry_query,entry_proximity):

    global result_box_positional  # Access result_box globally
    
    query = entry_query.get()
    proximity = int(entry_proximity.get())  # Convert proximity to integer
    
    pp_query = Pre_Processing_query(query)
    print(pp_query)
    
    porter_stemmer = PorterStemmer()
    
    words = pp_query.split()
    stemmed_query = []
    
    for word in words:
        stemmed_query.append(porter_stemmer.stem(word))
        
    print(stemmed_query)
    
    posting1 = inverted_index[stemmed_query[0]]
    posting2 = inverted_index[stemmed_query[1]]
    
    
    print(posting1)
    print("\n\n")
    print(posting2)
    
    result = []
    for dict1 in posting1:
        for dict2 in posting2:
            # Check if any key of dict1 exists in dict2
            for key1 in dict1:
                for key2 in dict2:
                    if key1  == key2:
                        print(f"Common key found: {key1}")
                        # Compare positions if the keys match
                        positions1 = dict1[key1]
                        print(positions1)
                        print("\n\n")
                        positions2 = dict2[key2]
                        print(positions2)
                        flag = False
                        for pos1 in positions1:
                            if flag is True:
                                break
                            flag = False    
                            for pos2 in positions2:
                                dist = abs(pos1 - pos2)
                                print(f"{pos1} - {pos2} = {dist}")
                                if dist <= proximity:
                                    result.append(key1)
                                    flag = True
                                    break
                                else:
                                    continue
            else:
                continue
            
    print(f"Result: {result}") 
    result_box_positional.delete(1.0, tk.END)  # Clear previous result
    result_box_positional.insert(tk.END, f"Result: {result}")
    
    
def main():
    global entry_query, result_box_inverted , result_box_positional ,entry_query
    
    root = tk.Tk()
    root.title("Query Processing GUI")
    
    directory = r'C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\ResearchPapers\\'
    directory1 = r'C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\'
    
    # Function to choose between Inverted Index and Positional Index
    def choose_index():
        global inverted_index,result_box_inverted,result_box_positional
        choice = int(choice_var.get())
        inverted_index = {}
        if choice == 1:
            inverted_index_window = tk.Toplevel(root)
            inverted_index_window.title("Inverted Index Window")
            for filename in os.listdir(directory):
                filepath = os.path.join(directory, filename)
                if os.path.isfile(filepath):
                    pp_filename = f"pp-{filename}"  # Name of the pre processed file
                    pp_filepath = os.path.join(directory1, pp_filename)

                    Pre_Processing(filepath, pp_filepath)

                    stem_filename = f"stemmed-{filename}"  # Name of the stem file
                    stem_filepath = os.path.join(directory1, stem_filename)

                    tokenize_and_stemm_doc_and_inserting_terms_in_Inverted_index(pp_filepath ,stem_filepath, int(filename.split('.')[0])) 

            print("Inverted Index:")
            for term, posting_list in inverted_index.items():
                print(term, ":", posting_list)
            inverted_index_filepath = os.path.join(directory1 ,'inverted_index.txt' )
            
            with open(inverted_index_filepath, 'w') as file:
                for term , posting_list in inverted_index.items():
                    file.write(f"{term}: {posting_list}\n")
            print("Inverted Index written to file successfully!")   
            
            # Query entry
            label_query = tk.Label(inverted_index_window, text="Enter your Query:")
            label_query.pack()
            entry_query = tk.Entry(inverted_index_window, width=50)
            entry_query.pack()
            btn_process_query = tk.Button(inverted_index_window, text="Process Query", command=lambda: query_processing(entry_query))
            btn_process_query.pack()
             # Result display
            label_result_inverted = tk.Label(inverted_index_window, text="Result:")
            label_result_inverted.pack()
            result_box_inverted = scrolledtext.ScrolledText(inverted_index_window, width=60, height=10)
            result_box_inverted.pack()

        elif choice == 2:
            positional_index_window = tk.Toplevel(root)
            positional_index_window.title("Positional Index Window")
            for filename in os.listdir(directory):
                filepath = os.path.join(directory, filename)
                if os.path.isfile(filepath):
                    pp_filename = f"pp-{filename}"  # Name of the pre processed file
                    pp_filepath = os.path.join(directory1, pp_filename)

                    Pre_Processing(filepath, pp_filepath)

                    stem_filename = f"stemmed-{filename}"  # Name of the stem file
                    stem_filepath = os.path.join(directory1, stem_filename)

                    tokenize_and_stemm_doc_and_inserting_terms_in_Positional_Inverted_index(pp_filepath ,stem_filepath, int(filename.split('.')[0])) 
            print("Inverted Index:")
            for term, posting_list in inverted_index.items():
                print(term, ":", posting_list)
                
            inverted_index_filepath = os.path.join(directory1 ,'Positional_inverted_index.txt' )
            with open(inverted_index_filepath, 'w') as file:
                for term , posting_list in inverted_index.items():
                    file.write(f"{term}: {posting_list}\n")
            print("Inverted Index written to file successfully!")
            label_query = tk.Label(positional_index_window, text="Enter your Query:")
            label_query.pack()
            entry_query = tk.Entry(positional_index_window, width=50)
            entry_query.pack()

            # Proximity entry
            label_proximity = tk.Label(positional_index_window, text="Enter proximity:")
            label_proximity.pack()
            entry_proximity = tk.Entry(positional_index_window, width=10)
            entry_proximity.pack()
            btn_proximity_query = tk.Button(positional_index_window, text="Proximity Query", command=lambda: proximity_query(entry_query, entry_proximity))
            btn_proximity_query.pack()
             # Result display
            label_result_positional = tk.Label(positional_index_window, text="Result:")
            label_result_positional.pack()
            result_box_positional = scrolledtext.ScrolledText(positional_index_window, width=60, height=10)
            result_box_positional.pack()
                
        else:
            error_window = tk.Toplevel(root)
            error_window.title("Invalid option")
    
    # GUI elements for choosing between Inverted Index and Positional Index
    label_choice = tk.Label(root, text="Choose index type:")
    label_choice.pack()
    choice_var = tk.StringVar()
    radio_btn_inverted_index = tk.Radiobutton(root, text="Inverted Index", variable=choice_var, value=1)
    radio_btn_inverted_index.pack()
    radio_btn_positional_index = tk.Radiobutton(root, text="Positional Index", variable=choice_var, value=2)
    radio_btn_positional_index.pack()
    btn_choose_index = tk.Button(root, text="Choose", command=choose_index)
    btn_choose_index.pack()

   

    root.mainloop()

if __name__ == "__main__":
    main()


File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\ResearchPapers\\1.txt Pre-Processed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\pp-1.txt tokenized, stemmed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\stemmed-1.txt inserted in inverted index successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\ResearchPapers\\11.txt Pre-Processed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\pp-11.txt tokenized, stemmed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\stemmed-11.txt inserted in inverted index successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\ResearchPapers\\12.txt Pre-Processed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\pp-12.txt tokenized, stemmed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\stemmed-12.txt inserted in inverted index successfully!
File C:\Users\Ra

directlyselectstheglob : [22]
discreteandhardtosolv : [22]
discretedata : [22]
discreterandomvari : [22]
discretevari : [22]
discrimi : [22]
discrimina : [22]
discriminativefeatureselect : [22]
discussionabout : [22]
disjoint : [22]
disr : [22]
dissert : [22, 25, 26]
dissimilar : [22]
distancesynthesismethod : [22]
distrust : [3, 22]
disveri : [22]
dmdenotethefeaturedi : [22]
dongxu : [22]
dongyoungnoh : [22]
dongyun : [22]
doriginalfeatur : [22]
doubl : [22]
dri : [22]
drutsa : [22]
dsasd : [22]
dscipi : [22]
dst : [22]
dstare : [22]
dualproblemof : [22]
dubourg : [22]
duda : [22, 24, 26]
duetal : [22]
dule : [22]
dundanc : [22]
dutkowski : [22]
dutkowskiandgambin : [22]
dw : [8, 22]
dwsw : [22]
dwt : [22]
dy : [9, 22, 24]
eachfeaturecanb : [22]
eachfeatureisrankedindividuallyregardless : [22]
eachinternalnodeinthetreeisassociatedwith : [22]
eachoth : [22]
eachtimewhenanewfeatureisnotinclud : [22]
eachtimewhenatraininginst : [22]
eachviewand : [22]
ean : [22]
eastlans : [22]
eb : [22]

cancer AND learning
['cancer', 'and', 'learn']
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\ResearchPapers\\1.txt Pre-Processed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\pp-1.txt tokenized, stemmed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\stemmed-1.txt inserted in inverted index successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\ResearchPapers\\11.txt Pre-Processed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\pp-11.txt tokenized, stemmed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\stemmed-11.txt inserted in inverted index successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\ResearchPapers\\12.txt Pre-Processed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\pp-12.txt tokenized, stemmed successfully!
File C:\Users\Rana Wahaj\Desktop\IR theory\ResearchPapers1\\stemmed-12.txt inserted i

past : [{1: [1425, 36377, 39592, 41534, 43157]}, {12: [278, 27031, 50173, 52775, 65371, 65888, 66133, 68654, 72166, 76200, 77561, 129354, 129746]}, {13: [41985, 63513]}, {14: [41985, 63513]}, {15: [182, 1793]}, {16: [1651, 18522, 21361, 24505, 42850, 50309]}, {2: [12514]}, {21: [1984, 23148, 40510]}, {22: [11578, 16665, 139021]}, {7: [55841, 100235, 103851]}, {8: [3446, 37543]}]
path : [{1: [39648]}, {12: [12358, 33632]}, {13: [45476]}, {14: [45476]}, {2: [24664]}, {21: [11180, 16201]}, {3: [3763]}, {7: [65804, 70164, 108752, 112124]}]
patient : [{1: [17666, 18078]}, {11: [135, 264, 633, 1176, 2350, 2620, 3052, 3117, 3261, 3366, 3931, 4572, 4786, 5611, 5805, 6289, 6401, 6606, 8345, 8562, 8888, 9537]}, {12: [17662, 94361]}, {16: [48748, 75643]}, {2: [4219, 11236, 11338, 15760, 15823, 16323]}, {26: [32243, 32419, 41411]}, {3: [8377, 8708]}, {8: [183, 485, 1294, 1794, 5437, 11243, 12544, 13202, 13534, 14882, 15982, 16193, 16310, 16874, 17039, 17304, 17636, 18883, 19265, 19337, 19573, 1983

artificial intelligence
['artifici', 'intellig']
[{1: [49, 503, 1342, 1782, 4599, 5108, 70939, 71212, 71461, 75529, 76111, 77147, 77360, 77929, 78485, 78748, 79419, 79492, 79583, 81283, 81406, 82084, 84568, 84626, 86479, 88349, 89920, 90834, 91652]}, {12: [41956, 60580, 110044, 110287, 111134, 113258]}, {23: [3357]}, {25: [27891]}, {8: [317, 740, 1431, 2045, 3405, 8119, 15673, 19142, 28644, 28893, 30017, 30140, 31064, 31177, 31424, 32562, 32857, 33460, 33563, 34900, 37467, 37628, 37923]}]



[{1: [60, 514, 1793, 2152, 3279, 3752, 4610, 5119, 56830, 67493, 70950, 71472, 72629, 73451, 74549, 75081, 75540, 76122, 77158, 77216, 77940, 78496, 78759, 79430, 79503, 79594, 81294, 81417, 81721, 82095, 82780, 83583, 83717, 84579, 84637, 88075, 88360, 89931, 90845, 91663]}, {15: [10569, 39622, 57107, 63235]}, {16: [38889, 40541, 60561, 64496]}, {17: [53762]}, {2: [48, 1451, 1560, 1773, 2029, 3288, 5327, 11667, 17454, 23400, 29259, 35513, 41111, 42867, 46687, 47286, 49754, 50011, 52334]}, {21: [24